In [55]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from scipy.ndimage.filters import uniform_filter1d

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bird-species-classification/test_data/test_data/brhkyt/D72_0475.jpg
/kaggle/input/bird-species-classification/test_data/test_data/brhkyt/D72_0473.jpg
/kaggle/input/bird-species-classification/test_data/test_data/brhkyt/D72_0479.jpg
/kaggle/input/bird-species-classification/test_data/test_data/brhkyt/D72_0477.jpg
/kaggle/input/bird-species-classification/test_data/test_data/brhkyt/D72_0474.jpg
/kaggle/input/bird-species-classification/test_data/test_data/brhkyt/D72_0478.jpg
/kaggle/input/bird-species-classification/test_data/test_data/cbrtsh/_D32_10318.jpg
/kaggle/input/bird-species-classification/test_data/test_data/cbrtsh/_D32_10313.jpg
/kaggle/input/bird-species-classification/test_data/test_data/cbrtsh/_D32_10311.jpg
/kaggle/input/bird-species-classification/test_data/test_data/cbrtsh/_D32_10312.jpg
/kaggle/input/bird-species-classification/test_data/test_data/cbrtsh/_D32_10317.jpg
/kaggle/input/bird-species-classification/test_data/test_data/cbrtsh/_D32_10314.jpg
/kag

/tmp/ipykernel_32/1581596860.py:11: DeprecationWarning: Please use `uniform_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import uniform_filter1d


In [56]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [57]:
train_data = ImageDataGenerator(rescale=(1./255),horizontal_flip=True,shear_range=0.2)
test_data = ImageDataGenerator(rescale=(1./255),horizontal_flip=True, shear_range=0.2)  #--> (0 to 255) convert to (0 to 1)
     

In [58]:
training = train_data.flow_from_directory('/kaggle/input/bird-species-classification/train_data/train_data',
                                      target_size=(240, 240),
                                      class_mode='categorical', 
                                      batch_size=16)
testing = test_data.flow_from_directory('/kaggle/input/bird-species-classification/test_data/test_data',
                                    target_size=(240, 240),
                                      class_mode='categorical', 
                                      batch_size=16)

Found 150 images belonging to 16 classes.
Found 157 images belonging to 16 classes.


In [59]:
training.class_indices

{'blasti': 0,
 'bonegl': 1,
 'brhkyt': 2,
 'cbrtsh': 3,
 'cmnmyn': 4,
 'gretit': 5,
 'hilpig': 6,
 'himbul': 7,
 'himgri': 8,
 'hsparo': 9,
 'indvul': 10,
 'jglowl': 11,
 'lbicrw': 12,
 'mgprob': 13,
 'rebimg': 14,
 'wcrsrt': 15}

In [60]:
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.models import Sequential

In [61]:

model = Sequential()
model.add(Convolution2D(20,(3,3),activation='relu',input_shape=(120, 120, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(45,activation='relu'))
model.add(Dense(16,activation='softmax'))
     

In [62]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [63]:
model.fit(train,batch_size=8,validation_data=test,epochs=10)

Epoch 1/10
19/19 [==============================] - 51s 3s/step - loss: 5.3304 - accuracy: 0.1000 - val_loss: 2.8024 - val_accuracy: 0.0701
Epoch 2/10
19/19 [==============================] - 49s 3s/step - loss: 2.7269 - accuracy: 0.0733 - val_loss: 2.8068 - val_accuracy: 0.0701
Epoch 3/10
19/19 [==============================] - 48s 3s/step - loss: 2.6052 - accuracy: 0.0800 - val_loss: 2.7249 - val_accuracy: 0.0637
Epoch 4/10
19/19 [==============================] - 62s 3s/step - loss: 2.5114 - accuracy: 0.0800 - val_loss: 2.8243 - val_accuracy: 0.0828
Epoch 5/10
19/19 [==============================] - 48s 3s/step - loss: 2.3806 - accuracy: 0.1600 - val_loss: 2.8386 - val_accuracy: 0.0828
Epoch 6/10
19/19 [==============================] - 49s 3s/step - loss: 2.3670 - accuracy: 0.2200 - val_loss: 2.8826 - val_accuracy: 0.0701
Epoch 7/10
19/19 [==============================] - 49s 3s/step - loss: 2.3234 - accuracy: 0.2267 - val_loss: 2.8774 - val_accuracy: 0.0764
Epoch 8/10
19/19 [==

In [64]:
model.save('animalWeights.h5')

In [65]:
# Testing

import numpy as np
from tensorflow.keras.preprocessing import image

In [66]:
# Testing 1 --> WRONG
img1 = image.load_img('/kaggle/input/bird-species-classification/train_data/train_data/blasti/DSC_6383.jpg',target_size=(120,120))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ['blasti','bonegl','brhkyt','cbrtsh','cmnmyn','gretit','hilpig','himbul','himgri','hsparo','indvul','jglowl','lbicrw','mgprob','rebimg','wcrsrt']
print(output[pred])

1/1 [==============================] - 0s 79ms/step
9
hsparo


In [67]:
# Testing 2 ---> WRONG
img2 = image.load_img('/kaggle/input/bird-species-classification/train_data/train_data/lbicrw/100_4037.JPG',target_size=(120,120))
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2,axis=0)
pred = np.argmax(model.predict(img2))
print(pred)
output = ['blasti','bonegl','brhkyt','cbrtsh','cmnmyn','gretit','hilpig','himbul','himgri','hsparo','indvul','jglowl','lbicrw','mgprob','rebimg','wcrsrt']
print(output[pred])

1/1 [==============================] - 0s 28ms/step
8
himgri


In [68]:
#MODEL TUNING

model = Sequential()
model.add(Convolution2D(12,(3,3),activation='relu',input_shape=(120, 120, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(24,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(36,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(62,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(16,activation='softmax'))

In [69]:

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 118, 118, 12)      336       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 59, 59, 12)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 57, 57, 24)        2616      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 28, 28, 24)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 26, 26, 36)        7812      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 13, 13, 36)      

In [70]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [71]:
model.fit(train,batch_size=8,validation_data=test,epochs=30)

Epoch 1/30
19/19 [==============================] - 51s 3s/step - loss: 2.7720 - accuracy: 0.0933 - val_loss: 2.7532 - val_accuracy: 0.1401
Epoch 2/30
19/19 [==============================] - 61s 3s/step - loss: 2.7354 - accuracy: 0.1000 - val_loss: 2.7472 - val_accuracy: 0.1529
Epoch 3/30
19/19 [==============================] - 48s 3s/step - loss: 2.6793 - accuracy: 0.1267 - val_loss: 2.7245 - val_accuracy: 0.1274
Epoch 4/30
19/19 [==============================] - 49s 3s/step - loss: 2.5931 - accuracy: 0.1933 - val_loss: 2.7114 - val_accuracy: 0.1656
Epoch 5/30
19/19 [==============================] - 49s 3s/step - loss: 2.4873 - accuracy: 0.1933 - val_loss: 2.7063 - val_accuracy: 0.1720
Epoch 6/30
19/19 [==============================] - 49s 3s/step - loss: 2.5988 - accuracy: 0.1333 - val_loss: 2.7613 - val_accuracy: 0.0701
Epoch 7/30
19/19 [==============================] - 49s 3s/step - loss: 2.6745 - accuracy: 0.1200 - val_loss: 2.7413 - val_accuracy: 0.1529
Epoch 8/30
19/19 [==

In [72]:
# Testing 1 ---> CORRECT
img1 = image.load_img('/kaggle/input/bird-species-classification/train_data/train_data/hsparo/100_4758.JPG',target_size=(120,120))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ['blasti','bonegl','brhkyt','cbrtsh','cmnmyn','gretit','hilpig','himbul','himgri','hsparo','indvul','jglowl','lbicrw','mgprob','rebimg','wcrsrt']
print(output[pred])

1/1 [==============================] - 0s 104ms/step
9
hsparo


In [73]:
# Testing 2 ----> WRONG
img2 = image.load_img('/kaggle/input/bird-species-classification/train_data/train_data/gretit/100_5044.JPG',target_size=(120,120))
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2,axis=0)
pred = np.argmax(model.predict(img2))
print(pred)
output = ['blasti','bonegl','brhkyt','cbrtsh','cmnmyn','gretit','hilpig','himbul','himgri','hsparo','indvul','jglowl','lbicrw','mgprob','rebimg','wcrsrt']
print(output[pred])

1/1 [==============================] - 0s 23ms/step
9
hsparo


In [74]:
# Testing 3 ---> CORRECT
img3 = image.load_img('/kaggle/input/bird-species-classification/train_data/train_data/indvul/DSC_0572.jpg',target_size=(120,120))
img3 = image.img_to_array(img3)
img3 = np.expand_dims(img3,axis=0)
pred = np.argmax(model.predict(img3))
print(pred)
output = ['blasti','bonegl','brhkyt','cbrtsh','cmnmyn','gretit','hilpig','himbul','himgri','hsparo','indvul','jglowl','lbicrw','mgprob','rebimg','wcrsrt']
print(output[pred])

1/1 [==============================] - 0s 25ms/step
10
indvul


In [75]:
# Testing 3 ---> CORRECT
img4 = image.load_img('/kaggle/input/bird-species-classification/train_data/train_data/brhkyt/D72_0470.jpg',target_size=(120,120))
img4 = image.img_to_array(img4)
img4 = np.expand_dims(img4,axis=0)
pred = np.argmax(model.predict(img4))
print(pred)
output = ['blasti','bonegl','brhkyt','cbrtsh','cmnmyn','gretit','hilpig','himbul','himgri','hsparo','indvul','jglowl','lbicrw','mgprob','rebimg','wcrsrt']
print(output[pred])

1/1 [==============================] - 0s 24ms/step
2
brhkyt


Model tuning (with dropout, batch normalization & early stopping)

In [76]:
from tensorflow.keras.layers import BatchNormalization, Dropout
     

In [77]:
# Initializing the seq model
model = Sequential()
# Adding conv layer with input
model.add(Convolution2D(12,(3,3),activation='relu',input_shape=(120, 120, 3)))
# Normalizing the conv layer output
model.add(BatchNormalization())
# Selecting the max values
model.add(MaxPooling2D(pool_size=(2,2)))
# Dropping the unwanted 20% of data
model.add(Dropout(0.2))
model.add(Convolution2D(24,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Convolution2D(36,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
# Hiddern layers
model.add(Dense(62,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
# Output layer
model.add(Dense(16,activation='softmax'))
     

In [78]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 118, 118, 12)      336       
                                                                 
 batch_normalization_4 (Batc  (None, 118, 118, 12)     48        
 hNormalization)                                                 
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 59, 59, 12)       0         
 g2D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 59, 59, 12)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 57, 57, 24)        2616      
                                                                 
 batch_normalization_5 (Batc  (None, 57, 57, 24)      

In [79]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
     

In [80]:
# Early stopping
from tensorflow.keras.callbacks import EarlyStopping
     


In [81]:
early_stop = EarlyStopping(monitor='accuracy',patience=8)

In [82]:
model.fit(train,batch_size=8,validation_data=test,epochs=30,callbacks=early_stop)

Epoch 1/30
19/19 [==============================] - 53s 3s/step - loss: 2.7257 - accuracy: 0.1067 - val_loss: 2.7407 - val_accuracy: 0.0446
Epoch 2/30
19/19 [==============================] - 48s 3s/step - loss: 2.5385 - accuracy: 0.2133 - val_loss: 2.8202 - val_accuracy: 0.0382
Epoch 3/30
19/19 [==============================] - 49s 3s/step - loss: 2.3879 - accuracy: 0.3000 - val_loss: 2.8831 - val_accuracy: 0.1083
Epoch 4/30
19/19 [==============================] - 50s 3s/step - loss: 2.2717 - accuracy: 0.3067 - val_loss: 3.4485 - val_accuracy: 0.0955
Epoch 5/30
19/19 [==============================] - 48s 3s/step - loss: 2.2776 - accuracy: 0.3133 - val_loss: 3.3990 - val_accuracy: 0.0955
Epoch 6/30
19/19 [==============================] - 50s 3s/step - loss: 2.1550 - accuracy: 0.3533 - val_loss: 3.1512 - val_accuracy: 0.0955
Epoch 7/30
19/19 [==============================] - 49s 3s/step - loss: 1.9861 - accuracy: 0.3800 - val_loss: 3.8990 - val_accuracy: 0.0955
Epoch 8/30
19/19 [==

In [83]:
# Testing 1 ---> WRONG
img2 = image.load_img('/kaggle/input/bird-species-classification/train_data/train_data/cmnmyn/DSCN5784.jpg',target_size=(120,120))
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2,axis=0)
pred = np.argmax(model.predict(img2))
print(pred)
output = ['blasti','bonegl','brhkyt','cbrtsh','cmnmyn','gretit','hilpig','himbul','himgri','hsparo','indvul','jglowl','lbicrw','mgprob','rebimg','wcrsrt']
print(output[pred])

1/1 [==============================] - 0s 182ms/step
8
himgri


In [87]:
# Testing 2 ---> CORRECT
img2 = image.load_img('/kaggle/input/bird-species-classification/test_data/test_data/himgri/IMG_5386.JPG',target_size=(120,120))
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2,axis=0)
pred = np.argmax(model.predict(img2))
print(pred)
output = ['blasti','bonegl','brhkyt','cbrtsh','cmnmyn','gretit','hilpig','himbul','himgri','hsparo','indvul','jglowl','lbicrw','mgprob','rebimg','wcrsrt']
print(output[pred])

1/1 [==============================] - 0s 26ms/step
8
himgri
